<img src="images/cs5228-header-title.png" />

# Assignment 4 - Linear Models, Recommender Systems & Graph Mining (50 Points)

Hello everyone, this assignment notebook covers Linear Models, Recommender Systems & Graph Mining. There are some code-completion tasks and question-answering tasks in this answer sheet. For code completion tasks, please write down your answer (i.e., your lines of code) between sentences that "Your code starts here" and "Your code ends here". The space between these two lines does not reflect the required or expected lines of code. For answers in plain text, you can refer to [this Markdown guide](https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd) to customize the layout (although it shouldn't be needed).

When you work on this notebook, you can insert additional code cells (e.g., for testing) or markdown cells (e.g., to keep track of your thoughts). However, before the submission, please remove all those additional cells again. Thanks!

**Important:** 
* Remember to rename and save this Jupyter notebook as **A4_YourName_YourNUSNETID.ipynb** (e.g., **A4_BobSmith_e12345678.ipynb**) before submission!
* Remember to rename and save the Python script file **A4_YourName_YourNUSNETID.py** (e.g., **A4_BobSmith_e12345678.py**) before submission!
* Submission deadline is **Nov 07, 11.59 pm**. Late submissions will be penalized by 10% for each additional day. Failure to appropriately rename both files will yield a penalty of 1 Point. There is no need to use your full name if it's rather long; it's just important to easily identify you in Canvas etc.

Please also add your NUSNET and student id in the code cell below. This is just to make any identification of your notebook doubly sure.

In [ ]:
student_id = ''
nusnet_id = ''

Here is an overview over the tasks to be solved and the points associated with each task. The notebook can appear very long and verbose, but note that a lot of parts are there to provide additional explanations, documentation, or some discussion. The code and markdown cells you are supposed to complete are well marked, but you can use the overview below to double-check that you covered everything.

* **1 Logistic Regression (20 Points)**
    * 1.1 Implementing Logistic Regression (12 Points)
        * 1.1 a) Calculating the Gradient (4 Points)
        * 1.1 b) Implementing Gradient Descent (3 Points)
        * 1.1 c) Predicting Labels (2 Points)
        * 1.1 d) Hyperparameter Tuning "By Hand" and Interpretation of Results (3 Points)
    * 1.2 Questions about Logistic Regression (8 Points)
* **2 Recommender Systems (20 Points)**
    * 2.1 Matrix Factorization (15 Points)
        * 2.1 a) Implement Method `fit()` (8 Points)
        * 2.1 b) Hyperparameter Exploration (2 Points)        
        * 2.1 c) Matrix Factorization & Updates (5 Points)
    * 2.2 Questions about Recommender Systems (5 Points)
* **3 Graph Mining: Community Detection (10 Points)**
    * 3.1 Implement Girvan-Newman Algorithm (4 Points)
    * 3.2 Question about Girvan-Newman Algorithm (6 Points)

## Setting up the Notebook

In [ ]:
# Some magic so that the notebook will reload the external python script file any time you edit and save the .py file;
%load_ext autoreload
%autoreload 2

Making all the required imports:

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import matplotlib.pyplot as plt
from src.utils import plot_mrt_graph

np.set_printoptions(precision=3)

**Important:** This notebook also requires you to complete in a separate `.py` script file. This keeps this notebook cleaner and simplifies testing your implementations for us. As you need to rename the file `A4.py`, you also need to edit the import statement below accordingly.

In [ ]:
from A4_SOLUTION import MyLogisticRegression, NMF, girvan_newman
#from A4_BobSmith_e12345678 import MyLogisticRegression, NMF, girvan_newman # <-- you will need to rename this accordingly

---

## 1 Logistic Regression

### Dataset Preparation & Visualization (nothing for you to do here)

For this task, we use the [Banknote Authentication Dataset](https://archive.ics.uci.edu/ml/datasets/banknote+authentication). It contains 1,371 samples, each with 4 numerical features (variance, skewness, kurtosis, and entropy of an image) to predict whether a bank note is genuine or a forgery. We use this dataset mainly for convenience. It basically does not require any data preprocessing as all features are numerical, and there are no "dirty" records. There is also no need for normalization/standardization as all features values are about the same range.

**Load & Prepare Dataset.** First, let's load the dataset into a pandas dataframe and have a quick look:

In [ ]:
df_bank = pd.read_csv('data/a4-banknote-authentication.csv')

df_bank.head()

For further processing, we convert the data into numpy arrays; pandas provides convenient methods for that. We also split the data into the input features and the target variable, with the latter being the last column in our data matrix. Lastly, we use `train_test_split` from scitkit-learn to split our dataset into the training and test data. We set the `random_state` to ensure consistent results throughout this task.

In [ ]:
# Convert pandas dataframe into numpy array
data = df_bank.to_numpy()

# Separate input features from target variable
X_bank = data[:,:-1]
y_bank = data[:,-1].astype(int)

# Split into training and test set; we use a common 80/20 split here
X_bank_train, X_bank_test, y_bank_train, y_bank_test = train_test_split(X_bank, y_bank, test_size=0.2, random_state=0)

**Visualization.** It never hurts to first have a quick look at the data. Since we have 4 input features, plotting the data points based on all features is not really possible. What we can do however, since we only have 4 features, we can generate the plots for all *pairs* of features. The code cell below accomplishes this.

In [ ]:
indices_0 = np.where(y_bank==0)
indices_1 = np.where(y_bank==1)

features = ['Variance', 'Skewness', 'Kurtosis', 'Entropy']

for feautre_1 in range(X_bank.shape[1]):
    for feature_2 in range(feautre_1+1, X_bank.shape[1]):
        plt.figure()
        plt.scatter(X_bank[:,feautre_1][indices_0], X_bank[:,feature_2][indices_0], c='blue')
        plt.scatter(X_bank[:,feautre_1][indices_1], X_bank[:,feature_2][indices_1], c='red')
        plt.xlabel(features[feautre_1], fontsize=16)
        plt.ylabel(features[feature_2], fontsize=16)
        plt.tight_layout()
        plt.show()

From the plots we can see that some of the input features seem to be quite helpful in separating the two classes. For example, if we would only use Variance and Skewness (first plot), we could probably get out a decent classifier. In contrast, considering only Kurtosis and Entropy (last plot), we see only a very poor separation of the class labels, so we would not be able to train a good classifier. Summing up, since already some subsets of features seem to be quite useful, we can expect that considering all features will potentially yield very good results.

**Training an off-the-shelf Logistic Regression model.** Before you will implement and test your own Logistic Regression classifier, let's first use [`sklearn.linear_model.LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). This will give us a first idea what accuracy result we can aim for.

In [ ]:
# Train Logistic Regression model over training data
sk_logreg = LogisticRegression().fit(X_bank_train, y_bank_train)

# Predict labels for test data
y_bank_sk_pred = sk_logreg.predict(X_bank_test)

# Print results (i.e., comparing predicted labels with groundtruth labels of test data)
print('f1 score (test): {:.3f}'.format(f1_score(y_bank_sk_pred, y_bank_test)))

# Show number of iterations (for a later comparison)
print('Number of iterations: {}'.format(sk_logreg.n_iter_[0]))

The results indeed confirm that we can expect to get very good results with our own implementation.

### 1.1 Implementing Logistic Regression (14 Points)

Your task here is to implement a Logistic Regression Classifier for binary classification. One learning outcome is actually to show how quick and easy this is. Below you can find the skeleton code for the class implementing the Logistic Regression Classifier. You will need to complete this code step by step along with the subtasks 2.1 a-c). 

We give you two basic methods for free so you can focus on the core parts:

* `add_bias()` adds the constant feature value $x_0 = 1$ for each data sample to implement the "bias trick"
* `calc_closs()` computes the Cross Entropy loss for the binary case, i.e., directly implementing the formula below

$$ L = -\frac{1}{n} \sum_{i=1}^n \left[ y_i\log{\hat{y}_i} + (1-y_i)\log{(1- \hat{y}_i} )\right]$$

#### 1.1 a) Calculating the Gradient (4 Points)

Given the non-linear nature of the loss function for the Logistic Regression, we cannot find the best $\theta$ values that minimize the loss function analytically. Hence, we have to implement it using Gradient Descent. Here, instead of calculating the $\nabla_\theta L$, setting it to 0, and solving for $\theta$, we start with initial parameter values for $\theta$, calculate the respective gradient, and update $\theta$ to reduce the loss $L$ iteratively.

The gradient for loss $L$ w.r.t. to $\theta$ is given as:

$$\nabla_\theta L = \frac{2}{n} X^T(h_\theta(X) - y)\ \text{, with }\ h_\theta(x_i) = \frac{1}{1+ e^{-\theta^{T}x_i}}$$


**Implement the methods `calc_h()` and `calc_gradient()` to calculate the gradient!** The only reason for doing this using 2 methods is that it allows us to re-use some code later on.

You can use the code cell below to check your implementation.

In [ ]:
np.random.seed(1)

my_logreg = MyLogisticRegression()

# Implement bias trick by adding x0 to all data samples
X_bank_train_with_bias = my_logreg.add_bias(X_bank_train)

# Initialize theta with random values
my_logreg.theta = np.random.rand(X_bank_train_with_bias.shape[1])

# Calculate the output of h(x)
h = my_logreg.calc_h(X_bank_train_with_bias)

# Calculate the gradient
grad = my_logreg.calc_gradient(X_bank_train_with_bias, y_bank_train, h)

print('theta values =\n{}\n\nGradient =\n{}\n'.format(my_logreg.theta, grad))

print('theta.shape = {} \t grad.shape = {}'.format(my_logreg.theta.shape, grad.shape))

#### 1.1 b) Implementing Gradient Descent (3 Points)

We now have everything in place to implement the training loop for Gradient Descent.

**Implement the method `fit()` to find the best $\theta$ using Gradient Descent!** Most of the code is given, so you can focus on the loop that performs Gradient Descent -- which is essentially utilizing the methods you have implemented in 1 a). You can test your implementation using the code below. 

In [ ]:
np.random.seed(0)

my_logreg = MyLogisticRegression().fit(X_bank_train, y_bank_train, lr=0.001, num_iter=100, verbose=True)

For the default hyperparameter values (`lr=0.01` and `num_iter=100`), the loss should go down to around 1.78.

#### 1.1 c) Predicting Labels (2 Points)

Now that you can train your Logistic Regression classifier, you only need to implement the prediction of class labels for unseen data samples.

**Implement the method predict()** to predict the class label (0 or 1) for an array of unseen data samples. 

In [ ]:
np.random.seed(0)

# Default values for 1.1 c)
lr = 0.001
num_iter = 100

# Your values you found for 1.1 d)
#lr = ...
#num_iter = ...


my_logreg = MyLogisticRegression().fit(X_bank_train, y_bank_train, lr=lr, num_iter=num_iter, verbose=True)

y_bank_my_pred = my_logreg.predict(X_bank_test)

y_bank_my_pred_train = my_logreg.predict(X_bank_train)
y_bank_my_pred_test = my_logreg.predict(X_bank_test)

f1_bank_train = f1_score(y_bank_train, y_bank_my_pred_train)
f1_bank_test = f1_score(y_bank_test, y_bank_my_pred_test)

print('f1 score (training): {:.3f}'.format(f1_bank_train))
print('f1 score (test): {:.3f}'.format(f1_bank_test))

For the default hyperparameter values (`lr=0.001` and `num_iter=100`), you should see f1 scores like below:

```
f1 score (training): 0.382
f1 score (test): 0.412
```

#### 1.1 d) Hyperparameter Tuning "By Hand" and Interpretation of Results (3 Points)

Seeing the loss going down (with the example parameter values above) is a good start. However, an f1 score over the test data if around 0.4 is not that great. In fact, we are worse than a random guesser. We also already know from using `sklearn` that an f1 score of over 0.98 is possible. Of course, we could simply increase the value of `num_iter` more and more, knowing that at some point Gradient Descent will eventually reach the minimum. But this would  unnecessarily increase the computation time.

**Try different values for `lr` and `num_iter` and briefly discuss your observations!** You can simply use the code cell above for playing around with both parameters. In more detail:

* Find a setting for both parameters that will increase the f1 score over the test data to at least 0.98; while keeping `num_iter` as small as possible. (Hint: You don't have to make fine-grained changes to the parameters. For example, there's no need in decreasing `num_iter` from 100 to 99 :). Overall, this should be very quick and easy to do. Just try some parameter combination and see how it behaves.

* Discuss any interesting observations you have made while finding such a parameter setting (e.g., how the development of the loss behaves, limitations on the choice of parameter values, etc.) together with a brief explanation. You might also want to compare the results/observations with the `sklearn` run above, as well as with the example we used in the lecture.

**Your Answer**:

### 1.2 Questions about Logistic Regression (8 Points)

**1.2 a) How would you need to modify/extend your implementation of `LogisticRegressionClassifier` to support Polynomial Logistic Regression? (2 Points)** There is no need for any implementation, just briefly describe any changes to your implementation that need to be made.

**Your Answer:**

Your `LogisticRegressionClassifier` class implements a binary classifier, i.e., the dataset has only two distinct class labels. However, you are now given a dataset of $c > 2$ distinct class labels -- that is, a multiclass classification task. For example, the [IRIS](https://archive.ics.uci.edu/ml/datasets/iris) dataset has 3 classes, i.e., $c=3$.

**1.2 b) Can you use your `LogisticRegressionClassifier` class to solve multiclass classification tasks? (2 Points)** If Yes, briefly describe the general approach how to implement this. If No, give a brief explanation why it is not possible.

**Your answer:**

Assume you want to implement our own class `MyLinearRegression` for solving Linear Regression Tasks using Gradient Descent. You could utilize the close relationship between Linear and Logistic Regression to simplify this task. In fact, the implementation of `MyLinearRegressionGD` would look very similar to the implementation of MyLogisticRegression, even simpler.

**1.2 c) Which method(s) of `MyLogisticRegression` need to be changed and how to implement `MyLinearRegression`? (4 Points)** You are free to do this task in two ways:

* **Option A:** Implement the methods using the code cell below (you don't have to run/test your code and, it may contain little syntax errors as long as the steps/calculations are clear; you may add comments as well, of course) **OR**
* **Option B:** Describe how the methods need to be changed in the Markdown cell below (please be provide sufficient detail to make the required changes and new calculations clear)

**Option A -- Your answer in Python code:**

In [ ]:
#########################################################################################
### Your code starts here ###############################################################



### Your code ends here #################################################################
#########################################################################################

**Option B -- Your Answer in Plain Text:**

---

## 2 Recommender Systems

Recommender systems are sophisticated algorithms designed to suggest relevant items to users by leveraging various data sources and techniques. These systems analyze patterns and preferences from user interactions, such as past purchases, browsing history, and ratings, to predict what users might like or find useful. Employed across numerous platforms, from e-commerce and streaming services to social media and news websites, recommender systems enhance user experience by personalizing content, thereby increasing engagement and satisfaction. Utilizing methods like collaborative filtering, content-based filtering, and hybrid approaches, these systems have become integral to navigating the vast amount of information available online efficiently.

### 2.1 Matrix Factorization (15 Points)

Matrix Factorization -- and here more specifically: non-negative Matrix Factorization -- is a class of algorithms where a matrix $M$ is factorized into (usually) two matrices $W$ and $H$, with the property that all three matrices have no negative elements. Matrix Factorization is popular techniques applied in recommender systems, where $W$ and $H$ contain a latent representation of all users and all items, respectively, and $M$ represents the rating matrix.

In this task, you will implement (non-negative) Matrix Factorization from scratch using Gradient Descent as covered in the lecture. In fact, we use the rating matrix $M$ which was used as an example in the lecture:

In [ ]:
M = np.array([
    [4, 0, 0, 5, 1, 0, 0],
    [5, 5, 4, 0, 0, 0, 0],
    [0, 0, 0, 2, 4, 5, 0],
    [0, 3, 0, 0, 0, 0, 3]
], dtype=float)

print(M)

We provide you with the skeleton code for class `NMF` (short for Non-Negative Matrix Factorization). The code includes the initialization of matrices `W` and `H`, as well as of Matrix `Z`. Matrix `Z` is an auxiliary matrix containing the indices of all non-zero entries of Matrix `M`. Recall from the lecture that we need to compute the Gradient Descent only based on the non-zero entries in the rating matrix.

The code cell below shows an example using the default parameter (`k=100`). The shapes of `W` and `H` reflect the number of users and items, as well as the size $k$ of the latent representations. The shape of `Z` is `(num_nonzero, 2)`. For example matrix `M`, the shape should be `(11, 2)` since `M` has 11 non-zero entries.

In [ ]:
np.random.seed(0)

nmf = NMF(M)

print('W.shape = {}'.format(nmf.W.shape))
print('H.shape = {}'.format(nmf.H.shape))
print('Z.shape = {}'.format(nmf.Z.shape))
print()
print('Z containing all the indices of all non-zero entries in M (first 5 entries only)')
print(nmf.Z[:5])

We also provide you with the method `calc_loss()` which calculates the loss w.r.t. the current values of matrices `W` and `H`. **Important:** Note that method implements the loss without regularization! Since we need this method only to print the loss and so to see its trend over time, this simplified calculation is sufficient.

In [ ]:
np.random.seed(0)

nmf = NMF(M)

loss = nmf.calc_loss()

print('Initial loss: {:.1f}'.format(loss))

#### 2.1 a) Implement Method `fit()`

Implement the class method `fit()` to perform matrix factorization using Gradient Descent! (8 Points).** The complete algorithm together with the required gradients is available as pseudo code in the lecture slides, and you are already familiar with the basic concept of Gradient Descent. Here, consider the regularization terms when calculating the gradients.

In [ ]:
np.random.seed(0)

nmf = NMF(M)

nmf.fit(verbose=True)

With the default values for all parameters  (`k=100`, `learning_rate=0.0001`, `lambda_reg=0.1`, `num_iter=100`), you should see a loss around **167.6** at the end of the training. *Note:* There are two slightly different alternatives for implementing Gradient Descent here, resulting in slightly different final losses. Both alternatives are fine. So if your final loss is around 165-168, it should be alright.

#### Predicting Unknown Ratings (nothing for you to do here)

With our learned estimates for `W` and `H`, we can calculate matrix `P` as the product of `W` and `H`, representing the matrix of predicted ratings. We encapsulate this simple computation in method `predict()`. Given its trivial implementation, we just give you this method here.

In [ ]:
P = nmf.predict()

print(np.around(P, 2))

With the default values for all parameters  (`k=100`, `learning_rate=0.0001`, `lambda_reg=0.1`, `num_iter=100`), the result should look something like this:

```
[[ 7.02 10.17 11.97  7.85  5.61 10.61 12.52]
 [ 7.75  6.9   8.05 11.22  9.09 14.9  13.09]
 [ 9.65  8.96 10.37  7.02  6.81  8.33 10.76]
 [ 9.11  7.25 10.69 11.67  9.07 12.4   9.27]]
```

*Note:* Again, given the two slighly different ways to implement Gradient Descent here, minor variations in output (e.g., in the 2nd decimal place) may occur and are perfectly fine.

#### 2.1 b) Hyperparameter Exploration (2 Points)

Explore different hyperparameter settings and briefly explain your observations! You can use the code cell for that; you can simply set different values for `k`, `learning_rate`, `lambda_reg`, and `num_iter`.

**Your Answer:**


In [ ]:
np.random.seed(0)

k, learning_rate, lambda_reg, num_iter = 100, 0.0001, 0.1, 100

nmf = NMF(M, k=k)

nmf.fit(learning_rate=learning_rate, lambda_reg=lambda_reg, num_iter=num_iter, verbose=True)

P = nmf.predict()

print('\nReconstructed rating matrix:')
print(np.around(P, 2))

#### 2.1 c) Matrix Factorization & Updates (5 Points)

You have now implemented a basic model-based recommender system using (non-negative) Matrix Factorization. Since we used only a toy rating matrix, performance was not an issue here. In real-world recommendations with many users and items, Matrix Factorization can be quite time consuming. The problem is that online platforms are very dynamic: users are joining and leaving, new items are added, users add new or update previous ratings. All of those cases change the rating matrix.

**How do different cases (e.g., new user/item/rating) affect a current result of a Matrix Factorization for a recommender system? (3 Points)** Outline the different problems, and discuss meaningful approaches to mitigate them. For example, a new user or item refers to the *Cold-Start Problem*. What are good practical strategies to address the Cold-Start Problem and other changes to the rating matrix using Matrix Factorization?

(Note: When you're discussing challenges regarding runtime/performance, please **exclude** any solutions relying on bigger clusters and parallel computing :). While those are valid points, in principle, here we want to focus on conceptual solutions).

**Your Answer:**

### 2.2 Questions about Recommender Systems (5 Points)

Assuming you are having access to the movie ratings dataset from both [IMDb](https://www.imdb.com/) and [Rotten Tomatoes](https://www.rottentomatoes.com/)

**2.2 a) Can you combine both datasets to make better recommendations, and if so, what are the requirements? (2 Points)** Hint: If you are not familiar with these two websites, first have a look how their basic movie rating works and differs.

**Your answer:**


Assume that you are building a Recommender System for movies using basic Collaborative Filtering. Now, if a new movie comes in, it won't have any ratings at the beginning, will therefore never be recommended and even less likely to get any ratings

**1.3 b) What are common strategies to mitigate this problem? (3 Points)** What are general strategies applicable to all types of items, and what are strategies particularly suitable for movies?

**Your answer:**

---

## 3 Graph Mining: Community Detection

### Load and Prepare Data (nothing for you to do here)

Throughout this section we work the MRT train network as our underlying graph. The MRT stations mark the nodes, and there is an edge (directed or undirected; see below) if there is a direct train connection between the respective MRT stations.

**Load data from files.** We first load the information about the MRT stations. We only need this information to have access to the latitude and longitude of the stations, so we can plot the MRT graph and preserve the relative geographic locations of the MRT stations.

**Note:** This dataset does not reflect the current state of the Singapore MRT network in 2024. Some of the more recently opened stations are missing. However, this is not important for the following task. Just to let you know in case someone cannot find his/her favorite station :).

In [ ]:
df_mrt_stations = pd.read_csv('data/a4-mrt-stations.csv')

df_mrt_stations.head()

The following file contains the main information: Which MRT stations are directly connected with by a train. Not that the file contains each connection twice for both directions.

In [ ]:
df_mrt = pd.read_csv('data/a4-mrt-connections.csv')

df_mrt.head()

**Create Graph.** From this data, we can easily create a NetworkX graph. Note that we treat the MRT network as an undirected graph here.

In [ ]:
G = nx.Graph()

for idx, row in df_mrt.iterrows():
    G.add_edge(row['to'], row['from'])

We provide you with the method `plot_mrt_graph()` to visualize the train network. As mentioned before, we can utilize the information about the geocoordinates of MRT stations to preserve their relative location. Of course the connections between the nodes / MRT stations are still just straight lines.

In [ ]:
plot_mrt_graph(G, df_mrt_stations)


### 3.1 Implement Girvan-Newman Algorithm (4 Points)

The Girvan-Newman Algorithm finds communities in a graph by assuming a strongly connected graph and then iteratively removing a minimum set of edges until the graph breaks into 2 components. The criteria to remove an edge is based on the Edge Betweenness Centrality; cf. lecture slides. The Edge Betweenness Centrality $c_{B}(e)$ of an Edge $e$ given a Graph $G=(V,E)$ is defined as:

$$c_{B}(e) = \sum_{u,w\in V} \frac{\sigma(v,w|e)}{\sigma(v,w)}$$

where $\sigma(v,w)$ is the number of shortest paths from $v$ to $w$, and $\sigma(v,w|e)$ is the number of shortest paths from $v$ to $w$ going through Edge $e$.

Similar to the more traditional Betweenness Centrality for nodes, Edge Betweenness Centrality also fundamentally requires solving the All-Pairs Shortest Paths (APSP) problem. As such, we could again utilize [`nx.algorithms.shortest_paths`](https://networkx.org/documentation/stable/reference/algorithms/shortest_paths.html). However, it gets a bit tedious since we here need **all** shortest paths between **all** pairs of nodes. So you can simply use [`nx.algorithms.centrality.edge_betweenness_centrality`](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.edge_betweenness_centrality.html) here.

**Implement the method `girvan_newman()` split a Graph G into 2 components!** You can assume that the Graph is undirected, unweighted, and strongly connected. Together with being able to use So you can simply use [`nx.algorithms.centrality.edge_betweenness_centrality`](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.edge_betweenness_centrality.html), this should make it a rather straightforward task. Here are 2 additional constraints:

* If 2 or more edges have the same maximum edge centrality, still remove only **1 edge** per iteration (you can randomly pick one of the edges with maximum edge centrality)
* Include a print statement that shows which edge has been removed in an iteration (see given code of method `girvan_newman()`); this is merely so you can check your implementation with the expected outcome below.


In [ ]:
communities, G_split = girvan_newman(G, verbose=True)

The expect output for the code cell above is:

```
Edge ('farrer road', 'botanic gardens') removed (edge betweenness centrality: 0.211)
Edge ('outram park', 'tiong bahru') removed (edge betweenness centrality: 0.217)
Edge ('harbourfront', 'outram park') removed (edge betweenness centrality: 0.245)
Edge ('marsiling', 'woodlands') removed (edge betweenness centrality: 0.380)
```

The returned Graph `G_split` is the original graph without the edges that needed to be removed to split the original Graph `G` into 2 components. As such, we can now plot `G_split` to visualize the 2 components. You need to zoom in to the MRT connections that reflect the removed edges.

In [ ]:
plot_mrt_graph(G_split, df_mrt_stations)

### 3.2 Question about Girvan-Newman Algorithm (6 Points)

This is a markdown cell. Please fill in your answers for (1)~(3), 2 Points each.

| No. | Question                                                                                                   |  You Answer |
|---------------------------------|--------------| ------- |
| (1)  | `girvan_newman()` removes the edge with the largest Edge Betweenness Centrality value. For the 4 edges that get removed from the MRT Graph, why do the values increase after each iteration? | ??? |
| (2)  | `girvan_newman()` always removes only one edge in each iteration even if 2 or more dges have the same maximum Edge Betweenness Centrality. Why is this meaningful, or why don't we remove all edge with the same maximum Edge Centrality in the same iteration? | ??? |
| (3)  | `girvan_newman()` resolves ties by randomly picking one of the edges with the maximum Edge Betweenness Centrality. Are there better/smarter ways to pick which edge to choose | ???   |